In [1]:
"""
To Run the Error Labeling Pipeline


"""

'\nTo Run the Error Labeling Pipeline\n\n\n'

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2020-11-29 04:16:03,335 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-29 04:16:03,337 - settings - Setting database.user to celiib
INFO - 2020-11-29 04:16:03,338 - settings - Setting database.password to newceliipass
INFO - 2020-11-29 04:16:03,341 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-29 04:16:03,342 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-29 04:16:03,353 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-29 04:16:03,648 - settings - Setting enable_python_native_blobs to True


# configuring the virtual module

In [3]:
test_mode = False

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 400)
print(f"Sleeping {random_sleep_sec} sec before conneting")

if not test_mode:
    time.sleep(random_sleep_sec)

print("Done sleeping")
import datajoint_utils as du
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
success_flag = False
for i in range(10):
    try:
        minnie,schema = du.configure_minnie_vm()
        
    except:
        print("Locked out trying agin in 30 seconds")
        time.sleep(30)
    else:
        success_flag = True
        
        
    if success_flag:
        print("successfully configured minnie")
        break

INFO - 2020-11-29 04:16:03,680 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-29 04:16:03,681 - settings - Setting database.user to celiib
INFO - 2020-11-29 04:16:03,682 - settings - Setting database.password to newceliipass
INFO - 2020-11-29 04:16:03,686 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-29 04:16:03,871 - settings - Setting enable_python_native_blobs to True


Sleeping 41 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
successfully configured minnie


# Defining Our Table

In [5]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [7]:
import error_detection as ed
ed = reload(ed)

In [8]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class AutoProofreadLabels(dj.Computed):
    definition="""
    -> minnie.Decomposition
    ---
    n_face_errors : int #the number of faces that were errored out
    face_idx_for_error : longblob #the face indices for the errors computed
    n_synapses: smallint unsigned #total number of synpases
    n_errored_synapses: smallint unsigned #the number of synapses
    """
    
    
    
    key_source = (minnie.Decomposition() & "n_somas = 1" & "n_faces>500000")
                  
    def make(self,key):
        global_start = time.time()
        segment_id = key["segment_id"]
        verbose = True
        
        print(f"\n\n----- Working on {segment_id}-------")
        whole_pass_time = time.time()
        
        #new method that checks if the information exists in the error table and if not then 
        error_table = (minnie.DecompositionErrorLabels() & dict(segment_id=segment_id))
        if len(error_table)>0:
            print("using quick fetch")
            current_mesh = du.fetch_segment_id_mesh(segment_id,minnie=minnie)
            returned_error_faces = error_table.fetch1("face_idx_for_error")
            
        else:
            neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")

            returned_error_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,visualize_errors_at_end=False)
            current_mesh = neuron_obj.mesh
            
        #------- Doing the synapse Exclusion Writing ---------- #
        data_to_write_new,n_synapses,n_errored_synapses = ed.get_error_synapse_inserts(current_mesh,
                                                                                       segment_id,
                                                                                       returned_error_faces,minnie=minnie,
                                                         return_synapse_stats=True,
                                                         verbose=True)
        
        if len(data_to_write_new)>0:
            print("Preparing to write errored synapses")
            minnie.SynapseExclude.insert(data_to_write_new,skip_duplicates=True)
            
        #------- Doing the Label Writing ---------- #
        new_key = dict(key,
                       n_face_errors = len(returned_error_faces),
                       face_idx_for_error = returned_error_faces,
                        n_synapses=n_synapses,
                        n_errored_synapses=n_errored_synapses)
        
        
        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)
        
        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
        

In [9]:
#(schema.jobs & "table_name='__decomposition_error_labels'").delete()
#minnie.SynapseExclude.delete()
#minnie.DecompositionErrorLabels.delete()

In [10]:
import time
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 900))
print('Populate Started')
if test_mode:
    AutoProofreadLabels.populate(reserve_jobs=True, suppress_errors=False)
else:
    AutoProofreadLabels.populate(reserve_jobs=True, suppress_errors=True)
print('Populate Done')

print(f"Total time for DecompositionErrorLabels populate = {time.time() - start_time}")

Populate Started


INFO - 2020-11-29 04:16:04,496 - autopopulate - Found 24895 keys to populate
INFO - 2020-11-29 04:16:04,579 - connection - Transaction started
INFO - 2020-11-29 04:16:04,581 - autopopulate - Populating: {'segment_id': 864691134939987555, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}




----- Working on 864691134939987555-------
using quick fetch


INFO - 2020-11-29 04:16:31,644 - connection - Transaction committed and closed.
INFO - 2020-11-29 04:16:31,652 - connection - Transaction started
INFO - 2020-11-29 04:16:31,654 - autopopulate - Populating: {'segment_id': 864691134939987811, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Processing 274 synapses
maximum mapping distance = 116.66028553980343
Number of errored synapses = (4,)
Preparing to write errored synapses


 ------ Total time for 864691134939987555 = 27.03312635421753 ------


----- Working on 864691134939987811-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f053532e488>, <function median_mesh_center at 0x7f053532e730>, <function n_spines at 0x7f053532e598>, <function n_faces_branch at 0x7f053532e2f0>, <function skeleton_distance_branch at 0x7f053532e510>, <function spines_per_skeletal_length at 0x7f053532e9d8>, <function no_spine_median_mesh_center at 0x7f053532e840>]
functions_list = [<function axon_segment at 0x7f05350ca0d0>]
Axons not keeping because of soma: {'L0': array([33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 56, 58, 65, 67, 68])}

----- Working on L1 ------
-- Axon Group 0 of size 25--
   Working on soma 0, starting_node 29
Not using 

InterfaceError: (0, '')